In [1]:

import cv2
import numpy as np

from datetime import datetime, timedelta

from skimage.measure import compare_ssim


In [2]:

cap = cv2.VideoCapture('../videos/scenes-01.mp4')

#
#    Flags for video I/O
#    https://docs.opencv.org/3.4/d4/d15/group__videoio__flags__base.html
#

frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))


print( 'Number of frames: %s' %  frame_count)
print( 'Video width x height: %s x %s' %  (frame_width, frame_height))
print( 'Frames per second (FPS): %s' %  fps)


Number of frames: 1920
Video width x height: 640 x 480
Frames per second (FPS): 29


In [3]:

def time_elapsed(millisec):

    time = str(timedelta(milliseconds=(millisec)))
    
    return time


In [4]:

# SPEED UP TEST SHOW

scale_movie = 10

small_width = int(frame_width/scale_movie)
small_height = int(frame_height/scale_movie)

frames_to_jump = 14

save_output = False

diff_treshold = 0.65


In [6]:


prev_frame = []

scene_counter = 1 
frames_examined = 0
frames_counter = 0

ret = True

if save_output:
    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('../videos/all-scenes-01.avi', fourcc, fps, (frame_width, frame_height))

while(ret):

    ret, frame = cap.read()

    if type(frame) is np.ndarray:
    
        if (frames_counter % frames_to_jump == 0): 
        
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            curr_frame = cv2.resize(gray, (small_width, small_height))

            if type(prev_frame) is np.ndarray:
                score = compare_ssim(prev_frame, curr_frame)

                if score < diff_treshold:
                    scene_counter = scene_counter + 1 
                    millisec = int(cap.get(cv2.CAP_PROP_POS_MSEC ))

                    print('New scene @frame :: @time -> %s :: %s' % (frames_counter, time_elapsed(millisec)))

            prev_frame = curr_frame
        
            frames_examined = frames_examined + 1

        frames_counter = frames_counter + 1
        
        cv2.putText(frame, 'Scene: #' + str(scene_counter), (40, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)    
        
        if save_output:
            out.write(frame)
    
cap.release()

if save_output:
    out.release()

print('')
print('Total number of frames: %s' % frames_counter)
print('Frames examined: %s' % frames_examined)
print('Number of scenes: %s' % scene_counter)


New scene @frame :: @time -> 182 :: 0:00:06.102000
New scene @frame :: @time -> 308 :: 0:00:10.304000
New scene @frame :: @time -> 462 :: 0:00:15.440000
New scene @frame :: @time -> 966 :: 0:00:32.248000
New scene @frame :: @time -> 1330 :: 0:00:44.387000

Total number of frames: 1920
Frames examined: 138
Number of scenes: 6
